# Peer-graded Assignment: Identifying neighborhoods to put a Mexican restaurant in Toronto

### Importing requiered libraries

In [1]:
import pandas as pd
import numpy as np

### Importing data and creating dataframe

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_df=pd.read_html(url, header=0)[0]
raw_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignoring cells without assigned borough

In [3]:
df = raw_df[raw_df["Borough"] != "Not assigned"].reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Grouping

In [4]:
df_grouped = df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assigning "Not Assigned" Neighbourhoods names the same name as the corresponding borough

In [5]:
df_grouped.loc[df_grouped['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df_grouped.loc[df_grouped['Neighbourhood'] == "Not assigned", 'Borough']
df_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Getting the shape of the dataframe

In [6]:
df_grouped.shape

(103, 3)

### Get coordinates into dataframe

In [7]:
url_cord = "http://cocl.us/Geospatial_data"
coordinates_df = pd.read_csv(url_cord)
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge coordinates into dataframe

In [8]:
df_merged = pd.merge(df_grouped, coordinates_df, on='Postal Code', how='inner')
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Install Folium to generate map

In [9]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: ...working... done

# All requested packages already installed.



### Generate map of Toronto´s neighborhoods

In [10]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



In [11]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer") 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [12]:

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map


In [13]:
CLIENT_ID = 'NGLIN0TXG3QHEQV4QLEFTIQ5LITWDBZ0FOH21MKFOPTRHH3L' # your Foursquare ID
CLIENT_SECRET = '14A0DHOD2XDZKEE5ND1S0YSF0GXHZ43ZTC2K3NCNDU5X4D0L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NGLIN0TXG3QHEQV4QLEFTIQ5LITWDBZ0FOH21MKFOPTRHH3L
CLIENT_SECRET:14A0DHOD2XDZKEE5ND1S0YSF0GXHZ43ZTC2K3NCNDU5X4D0L


In [14]:
import json
import requests

In [15]:
neighbourhood_name = df_merged.loc[0, 'Neighbourhood']
print(f"The first neighborhood's name is '{neighbourhood_name}'.")

The first neighborhood's name is 'Malvern, Rouge'.


In [16]:
neighbourhood_latitude = df_merged.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_merged.loc[0, 'Longitude'] # neighborhood longitude value
print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name , 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [17]:
radius = 5000
LIMIT = 100
url_fs = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
# get the result to a json file
results = requests.get(url_fs).json()
results

{'meta': {'code': 200, 'requestId': '5fbeba221492434970db8979'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 153,
  'suggestedBounds': {'ne': {'lat': 43.85168634500004,
    'lng': -79.13211520730404},
   'sw': {'lat': 43.76168625499995, 'lng': -79.25659159269598}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '542858a0498e22b7cfa91070',
       'name': 'Toronto Pan Am Sports Centre',
       'location': {'address': '875 Morningside Ave',
        'lat': 43.79062336247366,
        'lng': -79.19386919338744,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.79062

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
from pandas import json_normalize  # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
 
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
 
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
 
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
 
nearby_venues

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Morningside Park,Park,43.786546,-79.205322
...,...,...,...,...
95,Subway,Sandwich Place,43.771832,-79.185109
96,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.797597,-79.149406
97,Shoppers Drug Mart,Pharmacy,43.780599,-79.205009
98,Eggsmart,Breakfast Spot,43.797979,-79.149207


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

In [22]:
toronto_grouped = toronto_venues.groupby('Neighbourhood').count()
toronto_grouped.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22


In [23]:
toronto_grouped = toronto_venues.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Latitude,Venue Longitude
0,Agincourt,43.794200,-79.262029,43.792613,-79.261140
1,"Alderwood, Long Branch",43.602414,-79.543484,43.601445,-79.545006
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,43.755581,-79.440249
3,Bayview Village,43.786947,-79.385975,43.787903,-79.380860
4,"Bedford Park, Lawrence Manor East",43.733282,-79.419750,43.733578,-79.419342


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### List of grouped neighbordhoods where Restaurants are present, except Mexican restaurants

In [52]:
mex = toronto_grouped[(toronto_grouped["Mexican Restaurant"] == 0.0) & (toronto_grouped["Restaurant"] != 0.0)]
mex["Neighbourhood"]

2       Bathurst Manor, Wilson Heights, Downsview North
4                     Bedford Park, Lawrence Manor East
5                                           Berczy Park
7          Brockton, Parkdale Village, Exhibition Place
8     Business reply mail Processing Centre, South C...
13                                   Central Bay Street
14                                             Christie
18                       Commerce Court, Victoria Hotel
19                                           Davisville
21     Del Ray, Mount Dennis, Keelsdale and Silverthorn
22                                            Don Mills
28                         Fairview, Henry Farm, Oriole
29               First Canadian Place, Underground city
41                       India Bazaar, The Beaches West
48                             Little Portugal, Trinity
59                               Parkdale, Roncesvalles
64                             Richmond, Adelaide, King
68                                   Runnymede, 

### Using Foursquare data we were able to identify the neighborhoods that had restaurants, but no Mexican restaurants